In [ ]:
%cd /content/drive/MyDrive/ThesisData/hungngo/1-text2attribute_model

[Errno 2] No such file or directory: '/content/drive/MyDrive/ThesisData/hungngo/1-text2attribute_model'
/content


In [1]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
from dataclasses import dataclass, field
from typing import Optional

from transformers import (
    HfArgumentParser,
    TrainingArguments,
    set_seed
)
from transformers.trainer_utils import get_last_checkpoint
import json
import os

@dataclass
class TrainingArgs:
    dataset_config_name: Optional[str] = field(default=None)
    max_seq_length: int = field(default=256)
    overwrite_cache: bool = field(default=False)
    pad_to_max_length: bool = field(default=True)
    max_train_samples: Optional[int] = field(default=None)
    max_eval_samples: Optional[int] = field(default=None)
    max_predict_samples: Optional[int] = field(default=None)
    train_file: Optional[str] = field(default=None)
    validation_file: Optional[str] = field(default=None)
    test_file: Optional[str] = field(default=None)
    attributes: Optional[str] = field(default=None)
    num_labels: Optional[str] = field(default=None)
    output_dir: Optional[str] = field(default = "./tmp")


@dataclass
class ModelArguments:

    model_name_or_path: str = field()
    config_name: Optional[str] = field(default="bert-large-uncased")
    tokenizer_name: Optional[str] = field(default=None)
    cache_dir: Optional[str] = field(default=None)
    use_fast_tokenizer: bool = field(default=True)
    model_revision: str = field(default="main")
    use_auth_token: bool = field(default=False)
    ignore_mismatched_sizes: bool = field(default=False)






In [ ]:
!pip install datasets -q
!pip install transformers[torch] -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 55.7 MB/s eta 0:00:00


In [ ]:
do_train = True
do_valid = True
do_test = True
do_eval = True
pad_to_max_length = True

In [ ]:
from datasets import load_dataset
def load_data(data_files, cache_dir,auth_token):
  raw_datasets = load_dataset(
        "json",
        data_files=data_files,
        cache_dir=cache_dir,
        use_auth_token=True if auth_token else None,
    )
  return raw_datasets

In [ ]:
from collections import OrderedDict

def parser():
    # parser = HfArgumentParser((ModelArguments, TrainingArgs, TrainingArguments))
    # model_args, data_args, training_args = parser.parse_args_into_dataclasses()
    output_dir = "./tmp/"
    overwrite_output_dir = False
    last_checkpoint = get_last_checkpoint(output_dir)
    set_seed(42)
    data_files = {"train": "data/test.json", "validation": "data/test.json"}
    if os.path.isdir(output_dir) and do_train and not overwrite_output_dir:
      last_checkpoint = get_last_checkpoint(output_dir)
    raw_datasets = load_dataset(
        "json",
        data_files=data_files,
        cache_dir="./",
        use_auth_token=True,
    )
    num_labels = json.load(open("num_labels.json", 'r'))
    return raw_datasets, num_labels
raw_datasets,num_labels = parser()



FileNotFoundError: [Errno 2] No such file or directory: './tmp/'

In [ ]:
from transformers import (
    AutoTokenizer,
    BertConfig,
    BertTokenizer,
    BertTokenizerFast,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PretrainedConfig,
    Trainer,
    TrainingArguments,
    set_seed,
    EarlyStoppingCallback,
)
from mubert import BertForAttributModel
def load_model(config_name = "bert-large-uncased"):
  cache_dir ="./"
  config = BertConfig.from_pretrained(
        config_name,
        cache_dir=None,
        revision="main",
        use_auth_token=False,
    )
  tokenizer = AutoTokenizer.from_pretrained(
        config_name,
        cache_dir=None,
        use_fast=True,
        revision="main",
        use_auth_token=False,
    )
  model = BertForAttributModel.from_pretrained(
      config_name,
      config=config,
      num_labels = num_labels,
      tokenizer = tokenizer,
      cache_dir="./",
      revision="main",
      use_auth_token=False,
      ignore_mismatched_sizes=False,
  )
  special_tokens = [f"[unused{i}]" for i in range(len(num_labels))]
  if do_train:
    tokenizer.add_special_tokens({"additional_special_tokens":special_tokens})
    model.resize_token_embeddings(len(tokenizer))
  model.tokenizer = tokenizer

  return config, tokenizer, model
#model_name = "bert-base-multilingual-uncased"
config,tokenizer,model = load_model("./tmp/checkpoint-2000")


In [ ]:
max_seq_length = 256
pad_to_max_length = True
if pad_to_max_length:
  padding = "max_length"
else:
  padding = False
max_seq_length = min(max_seq_length, tokenizer.model_max_length)

In [ ]:
from copy import deepcopy
def preprocess_function(examples,attributes,tokenizer,padding,max_seq_length):
    result = tokenizer(examples['text'],
                        padding=padding,
                        max_length=max_seq_length,
                        truncation=True)
    if 'labels' in examples:
        for idx in range(len(examples['labels'])):
            att_value = OrderedDict()
            for order, att in enumerate(attributes):
                att_value[att] = examples['labels'][idx][order].index(1)
            examples['labels'][idx] = deepcopy(att_value)
        result['labels']= examples['labels']
    return result


In [ ]:
from functools import partial
attributes = json.load(open("data/att_key.json", 'r'))
attributes
raw_datasets = raw_datasets.map(
    partial(preprocess_function, tokenizer=tokenizer, attributes=attributes, padding = padding,max_seq_length= max_seq_length),
    batched=True,
    load_from_cache_file=True,
    desc="Running tokenizer on dataset",
)

In [ ]:
raw_datasets

In [ ]:
train_dataset = raw_datasets["train"]
# if data_args.max_train_samples is not None:
#     max_train_samples = min(len(train_dataset), data_args.max_train_samples)
#     train_dataset = train_dataset.select(range(max_train_samples))

In [ ]:
from data_collator import default_data_collator

if pad_to_max_length:
    data_collator = default_data_collator
elif training_args.fp16:
    data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=8)
else:
    data_collator = None

In [ ]:
from transformers import TrainingArguments

training_arguments = {
    "do_train": True,
    "do_eval": True,
    "per_device_train_batch_size": 16,
    "gradient_accumulation_steps": 4,
    "output_dir": "./tmp/",
    "load_best_model_at_end": True,
    "save_strategy": "steps",
    "evaluation_strategy": "steps",
    "num_train_epochs": 50,
    "overwrite_output_dir": True,
    "learning_rate": 1e-05

}

training_args = TrainingArguments(**training_arguments)



In [ ]:
eval_dataset = raw_datasets["validation"]


In [ ]:

trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset if training_args.do_train else None,
        eval_dataset=eval_dataset if training_args.do_eval else None,
        compute_metrics=None,
        tokenizer=tokenizer,
        data_collator=data_collator,
    )


In [ ]:
checkpoint = None

In [ ]:
import math
output_dir = "./tmp/"
overwrite_output_dir = False
last_checkpoint = get_last_checkpoint(output_dir)

set_seed(42)
if os.path.isdir(output_dir) and do_train and not overwrite_output_dir:
  last_checkpoint = get_last_checkpoint(output_dir)
if do_train:
    train_result = trainer.train(resume_from_checkpoint=checkpoint)
    metrics = train_result.metrics
    metrics["train_samples"] =  len(train_dataset)
    try:
        perplexity = math.exp(metrics["train_loss"])
    except OverflowError:
        perplexity = float("inf")
    metrics["train_perplexity"] = perplexity

    trainer.log_metrics("train", metrics)
    trainer.save_metrics("train", metrics)
    trainer.save_state()

In [ ]:
metrics = trainer.evaluate(eval_dataset=eval_dataset)
import math
max_eval_samples = len(eval_dataset)

metrics["eval_samples"] = min(max_eval_samples, len(eval_dataset))
try:
    perplexity = math.exp(metrics["eval_loss"])
except OverflowError:
    perplexity = float("inf")
metrics["eval_perplexity"] = perplexity

print(f"***** Eval metrics *****")
print(metrics)
trainer.save_metrics("eval", metrics)

In [ ]:
eval_dataset["labels"][0]

In [ ]:
predictions = trainer.predict(raw_datasets["validation"], metric_key_prefix="predict").predictions
result_output = {}
softmaxprobs = {}

In [ ]:
import numpy as np
import torch
for k,v in predictions.items():

    pred = np.argmax(predictions[k], axis=1)
    softmax = torch.nn.Softmax(dim=1)
    softmaxprobs[k] = softmax(torch.from_numpy(predictions[k])).tolist()
    result_output[k] = np.zeros(predictions[k].shape, dtype=np.int8)
    for p in range(len(pred)):
        result_output[k][p, pred[p]] = 1
    result_output[k] = result_output[k].tolist()

In [ ]:
eval_dataset["labels"]

In [ ]:
classes = []
for labels in result_output.keys():
  classes.append(labels)

### Evalulation

In [ ]:
labels_evals= {}
for ele in eval_dataset["labels"]:
  for v in ele.keys():
    if (v not in labels_evals.keys()):
      labels_evals[v] = []
    labels_evals[v].append(ele[v])


In [ ]:
eval = {}
for ele in classes:
  temp1 = labels_evals[ele]
  temp2 = result_output[ele]
  cnt = 0
  for i in range(len(temp1)):
    if (temp1[i] ==temp2[i].index(max(temp2[i]))):
      cnt = cnt+1
  eval[ele] = cnt*1.0/len(temp1)

In [ ]:
type(raw_datasets["validation"])

### Inference 1 sample

In [ ]:
import pandas as pd
import datasets
input_sentence = "Có đàn piano, không đàn guitar"

df = pd.DataFrame({'text': [input_sentence]})

dataset = datasets.Dataset.from_pandas(df)
def tokenize_function(examples):
    return tokenizer(examples['text'], truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True)
predictions = trainer.predict(tokenized_dataset, metric_key_prefix="predict").predictions
result_output = {}
for k,v in predictions.items():
  pred = np.argmax(predictions[k], axis=1)
  softmax = torch.nn.Softmax(dim=1)
  result_output[k] = np.zeros(predictions[k].shape, dtype=np.int8)
  for p in range(len(pred)):
      result_output[k][p, pred[p]] = 1
  result_output[k] = result_output[k].tolist()


In [ ]:
# 🔴🟠🟡🟢🔵🟣⚫️⚪️🟤🔺🔸🔹🔶🔷🟥🟧🟨🟩🟦🟪⬛️⬜️🟫

class ColorfulCharacter:
    BLUE_SQUARE = '\U0001F7E6'
    GREEN_SQUARE = '\U0001F7E9'
    RED_SQUARE = '\U0001F7E5'
    ORANGE_SQUARE = '\U0001F7E7'
    YELLOW_SQUARE = '\U0001F7E8'
    PURPLE_SQUARE = '\U0001F7EA'
    BLACK_SQUARE = '\U00002B1B'
    GRAY_SQUARE = '\U00002B1C'

class LabelValue:
    YES = [1, 0, 0]
    NO = [0, 1, 0]
    UNKNOWN = [0, 0, 1]

label_value_to_colorful_character_mapper = {
    str(LabelValue.YES): ColorfulCharacter.GREEN_SQUARE,
    str(LabelValue.NO): ColorfulCharacter.RED_SQUARE,
    str(LabelValue.UNKNOWN): ColorfulCharacter.BLACK_SQUARE
}

def label_value_to_colorful_character(label_value):
    if str(label_value) not in label_value_to_colorful_character_mapper:
        return label_value_to_colorful_character_mapper[
            str(label_value)
        ]
    else:
        return None

label_value_yes = [1, 0, 0]
label_value_no = [0, 1, 0]
label_value_unknown = [0, 0, 1]

print(f"{str(label_value_yes)}: {label_value_to_colorful_character(label_value_yes)}")
print(f"{str(label_value_no)}: {label_value_to_colorful_character(label_value_no)}")
print(f"{str(label_value_unknown)}: {label_value_to_colorful_character(label_value_unknown)}")


[1, 0, 0]: None
[0, 1, 0]: None
[0, 0, 1]: None


In [ ]:
# result_output

for k, v in result_output.items():
    print(f"\n{k}: ", end = ' ')
    for vec in v:
        print(label_value_to_colorful_character(label_value_yes), end=' ')

NameError: name 'result_output' is not defined

In [ ]:
eval

In [ ]:
json.dump(result_output, open(f"predict_attributes.json",'w'))
json.dump(softmaxprobs, open(f"softmax_probs.json",'w'))

In [ ]:
# result_output

for k, v in result_output.items():
    print(f"\n{k}: ", end = ' ')
    for vec in v:
        print(label_value_to_colorful_character(label_value_yes), end=' ')